In [58]:
from aocd import get_data, submit
# from aocd.get import current_day

from IPython.display import display
from IPython.core.display import Markdown

In [59]:
DAY = 5
YEAR = 2023

In [60]:
url = f"https://adventofcode.com/{YEAR}/day/{DAY}"
display(Markdown(f"#### See [{YEAR} Day {DAY}]({url})."))

#### See [2023 Day 5](https://adventofcode.com/2023/day/5).

In [61]:
data = get_data(year=YEAR, day=DAY)

In [62]:
data.splitlines()[:5]

['seeds: 3169137700 271717609 3522125441 23376095 1233948799 811833837 280549587 703867355 166086528 44766996 2326968141 69162222 2698492851 14603069 2755327667 348999531 2600461189 92332846 1054656969 169099767',
 '',
 'seed-to-soil map:',
 '404519244 291909660 232498813',
 '1320269465 1130502191 53088254']

# Part A

In [63]:
# data = """seeds: 79 14 55 13

# seed-to-soil map:
# 50 98 2
# 52 50 48

# soil-to-fertilizer map:
# 0 15 37
# 37 52 2
# 39 0 15

# fertilizer-to-water map:
# 49 53 8
# 0 11 42
# 42 0 7
# 57 7 4

# water-to-light map:
# 88 18 7
# 18 25 70

# light-to-temperature map:
# 45 77 23
# 81 45 19
# 68 64 13

# temperature-to-humidity map:
# 0 69 1
# 1 0 69

# humidity-to-location map:
# 60 56 37
# 56 93 4
# """

In [64]:
import re

In [65]:
def buildmap(start_row, stop_row):
    output = []
    for i in range(start_row+1, stop_row-1):
        l = data.splitlines()[i]
        to_start, from_start, length = map(int, l.split())
        output.append({
            'from_start': from_start,
            'from_end': from_start + length,
            'delta': to_start - from_start,
        })
    return output

def domap(n, input_map):
    for m in input_map:
        # print(m['from_start'])
        if (m['from_start'] <= n) and (n < m['from_end']):
            return n + m['delta']
    return n

sections = []
for i, l in enumerate(data.splitlines()):
    if 'map' in l:
        sections.append(i)
sections.append(len(data.splitlines())+1)

maps = []

for i in range(7):
    maps.append(buildmap(sections[i], sections[i+1]))

In [66]:
init_seeds = re.findall(r"(\d+)", data.splitlines()[0])
locations = []
for s in init_seeds:
    for m in maps:
        s = domap(int(s), m)
    locations.append(s)

In [67]:
ans = min(locations)
ans

388071289

In [11]:
submit(ans, year=YEAR, day=DAY)

answer a: 388071289
submitting for part b (part a is already completed)


AocdError: cowardly refusing to submit 388071289 for part b, because that was the answer for part a

# Part B

In [77]:
import portion as P

def buildmap(start_row, stop_row):
    output = []
    for i in range(start_row+1, stop_row-1):
        l = data.splitlines()[i]
        to_start, from_start, length = map(int, l.split())
        output.append({
            'interval': P.closed(from_start, from_start + length),
            'delta': to_start - from_start,
        })
    return output

def shift(interval, amount):
    result = P.empty()
    for i in interval:
        result |= P.closed(i.lower + amount, i.upper + amount)
    return result

sections = [i for i,l in enumerate(data.splitlines()) if 'map' in l]
sections.append(len(data.splitlines())+1)

steps = [buildmap(sections[i], sections[i+1]) for i in range(len(sections)-1)]

init_seeds = re.findall(r"(\d+)", data.splitlines()[0])

destination = P.empty()
for i in range(0,len(init_seeds),2):
    destination |= P.closed(int(init_seeds[i]), int(init_seeds[i]) + int(init_seeds[i+1]))

for step in steps:
    origin = destination
    destination = P.empty()
    for m in step:
        destination |= shift(m['interval'] & origin, m['delta'])
        origin = origin - m['interval']
    destination |= origin
ans = destination.lower
ans

84206669

In [ ]:
submit(ans, year=YEAR, day=DAY)